# Linear regression

We estimate simple linear regression model with a half-T prior.
First, we load the packages we use.

In [1]:
using StatisticalRethinking
using DynamicHMC, TransformVariables, LogDensityProblems, MCMCDiagnostics
using Parameters, ForwardDiff

ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

Import the dataset.

In [2]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults and standardize

In [3]:
df2 = filter(row -> row[:age] >= 18, df)
df2[:weight] = convert(Vector{Float64}, df2[:weight]);
df2[:weight_s] = (df2[:weight] .- mean(df2[:weight])) / std(df2[:weight]);
df2[:weight_s2] = df2[:weight_s] .^ 2;

Show the first six rows of the dataset.

In [4]:
first(df2, 6)

,height,weight,age,male,weight_s,weight_s2
,Float64⍰,Float64,Float64⍰,Int64⍰,Float64,Float64
1,151.765,47.8256,63.0,1,0.439097,0.192806
2,139.7,36.4858,63.0,0,-1.31718,1.73498
3,136.525,31.8648,65.0,0,-2.03287,4.13256
4,156.845,53.0419,41.0,1,1.24699,1.55498
5,145.415,41.2769,51.0,0,-0.575156,0.330804
6,163.83,62.9926,35.0,1,2.78812,7.77364


Then define a structure to hold the data: observables, covariates, and the degrees of freedom for the prior.

In [5]:
"""
Linear regression model ``y ∼ Xβ + ϵ``, where ``ϵ ∼ N(0, σ²)`` IID.
Flat prior for `β`, half-T for `σ`.
"""
struct LinearRegressionProblem{TY <: AbstractVector, TX <: AbstractMatrix,
Tν <: Real}
    "Observations."
    y::TY
    "Covariates"
    X::TX
    "Degrees of freedom for prior."
    ν::Tν
end

Main.##425.LinearRegressionProblem

Then make the type callable with the parameters *as a single argument*.

In [6]:
function (problem::LinearRegressionProblem)(θ)
    @unpack y, X, ν = problem   # extract the data
    @unpack β, σ = θ            # works on the named tuple too
    loglikelihood(Normal(0, σ), y .- X*β) + logpdf(TDist(ν), σ)
end

We should test this, also, this would be a good place to benchmark and
optimize more complicated problems.

In [7]:
N = size(df2, 1)
X = hcat(ones(N), hcat(df2[:weight_s], df2[:weight_s2]));
y = convert(Vector{Float64}, df2[:height])
p = LinearRegressionProblem(y, X, 1.0);
p((β = [1.0, 2.0, 3.0], σ = 1.0))

-4.001310714028722e6

For this problem, we write a function to return the transformation (as it varies with the number of covariates).

In [8]:
problem_transformation(p::LinearRegressionProblem) =
    as((β = as(Array, size(p.X, 2)), σ = asℝ₊))
# Wrap the problem with a transformation, then use Flux for the gradient.
P = TransformedLogDensity(problem_transformation(p), p)
∇P = ADgradient(:ForwardDiff, P);

Finally, we sample from the posterior. `chain` holds the chain (positions and
diagnostic information), while the second returned value is the tuned sampler
which would allow continuation of sampling.

In [9]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.0022 s/step ...done
MCMC, adapting ϵ (25 steps)
0.00066 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0027 s/step ...done
MCMC, adapting ϵ (100 steps)
0.00037 s/step ...done
MCMC, adapting ϵ (200 steps)
0.00022 s/step ...done
MCMC, adapting ϵ (400 steps)
0.00018 s/step ...done
MCMC, adapting ϵ (50 steps)
0.00029 s/step ...done
MCMC (1000 steps)
0.00024 s/step ...done


We use the transformation to obtain the posterior from the chain.

In [10]:
posterior = TransformVariables.transform.(Ref(∇P.transformation), get_position.(chain));
posterior[1:5]

5-element Array{NamedTuple{(:β, :σ),Tuple{Array{Float64,1},Float64}},1}:
 (β = [154.56, 6.45256, 0.148852], σ = 5.28369403566664)    
 (β = [154.496, 6.29349, -0.115618], σ = 5.172425987181745) 
 (β = [154.551, 5.5715, 0.0632081], σ = 4.993268526160111)  
 (β = [154.494, 5.70218, -0.10765], σ = 4.819265182603345)  
 (β = [155.021, 5.89168, -0.248484], σ = 5.0462605183232565)

Extract the parameter posterior means: `β`,

In [11]:
posterior_β = mean(first, posterior)

3-element Array{Float64,1}:
 154.6258765668        
   5.847670335408231   
  -0.013126210660355265

then `σ`:

In [12]:
posterior_σ = mean(last, posterior)

5.1079280677558145

Effective sample sizes (of untransformed draws)

In [13]:
ess = mapslices(effective_sample_size,
                get_position_matrix(chain); dims = 1)
# NUTS-specific statistics
NUTS_statistics(chain)

cmdstan_result = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
           Mean         SD       Naive SE       MCSE      ESS
    a 154.609019750 0.36158389 0.0057171433 0.0071845548 1000
   b1   5.838431778 0.27920926 0.0044146860 0.0048693502 1000
   b2  -0.009985954 0.22897191 0.0036203637 0.0047224478 1000
sigma   5.110136300 0.19096315 0.0030193925 0.0030728192 1000

Quantiles:
          2.5%        25.0%        50.0%       75.0%        97.5%
    a 153.92392500 154.3567500 154.60700000 154.8502500 155.32100000
   b1   5.27846200   5.6493250   5.83991000   6.0276275   6.39728200
   b2  -0.45954687  -0.1668285  -0.01382935   0.1423620   0.43600905
sigma   4.76114350   4.9816850   5.10326000   5.2300450   5.51500975
";

Extract the parameter posterior means: `β`,

In [14]:
[posterior_β, posterior_σ]

2-element Array{Any,1}:
  [154.626, 5.84767, -0.0131262]
 5.1079280677558145             

end of m4.5d.jl#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*